In [1]:
import pandas as pd
import sqlite3

What you need to do in this exercise (read the full task):

- create a connection to the database using the library sqlite3
- create a new table datamart in the database by joining the tables pageviews and
- checker using only one query

    - the table should have the following columns: uid, labname, first_commit_ts,
    first_view_ts
    - first_commit_ts is just a new name of the column timestamp from the checker
    table, it shows the first commit from a particular lab and from a particular
    user
    - first_view_ts is the first visit of a user to the table pageviews, timestamp
    when a user visited the newsfeed
    - status = ’ready’ should still be a filter
    - numTrials = 1 should still be a filter
    - labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’,
    ’laba06s’, ’project1’
    - the table should contain only the users (uids with user_*) and not the admins
    - first_commit_ts and first_view_ts should be parsed as datetime64[ns]


- using Pandas methods, create two dataframes: test and control

    - test should have the users that have the values in first_view_ts
    - control should have the users that have missing values in first_view_ts
    - replace the missing values in the control with the average first_view_ts of the
    test users, we will use this value for the future analysis
    - save both tables into the database, you will use them in the next exercises


- close the connection

In [67]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [61]:
query = """
CREATE TABLE datamart AS
SELECT checker.uid, 
        checker.labname, 
        checker.timestamp AS first_commit_ts, 
        MIN(pageviews.datetime) AS first_view_ts
FROM checker
LEFT JOIN pageviews USING(uid)
WHERE checker.status = 'ready' 
    AND checker.numTrials = 1 
    AND checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY checker.uid, checker.labname;
"""

conn.execute(query)

datamart = pd.io.sql.read_sql("SELECT * FROM datamart", conn, parse_dates=['first_commit_ts', 'first_view_ts'])

In [68]:
test_query = "SELECT * FROM datamart WHERE first_view_ts IS NOT NULL;"
test = pd.io.sql.read_sql(test_query, conn, parse_dates=['first_view_ts'])

In [63]:
control_query = "SELECT * FROM datamart WHERE first_view_ts IS NULL;"
control = pd.io.sql.read_sql(control_query, conn)
control['first_view_ts'] = test['first_view_ts'].mean()

In [64]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

81

In [70]:
conn.close()